In [ ]:
!pip install pandas
!pip install scikit-learn
!pip install pyproj

In [42]:
import numpy as np
import pandas as pd
import glob
import os
import pyproj
from sklearn.neighbors import NearestNeighbors

In [43]:
input_root_path = "P:\SALMALIS Lab\Work Zone Dataset"
weather_path = "E:\OneDrive - UBC\Work Zone\data\WeatherInfo"
output_root_path = "../"

In [44]:
def load_csv_to_df(files, filename=None, is_excel=False):
    if is_excel:
        df = pd.concat((pd.read_excel(f) for f in files), ignore_index=True)
    else:
        df = pd.concat((pd.read_csv(f, skipinitialspace = True) for f in files), ignore_index=True)

    if filename:
        df.to_csv(os.path.join(output_root_path, filename), header = True, index=True)
    return df

In [45]:
travel_t = pd.read_csv(os.path.join(input_root_path, 'TravelTime_HaveCellCoverage.csv'))
del travel_t[travel_t.columns[0]]
travel_t

C:\Users\yegmo\AppData\Local\Temp\ipykernel_18920\413896161.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  travel_t = pd.read_csv(os.path.join(input_root_path, 'TravelTime_HaveCellCoverage.csv'))


,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,...,Weather:Lat,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:CalculatedAADT,MaxSpeed
0,49831911074,Spread 3B,2022-05-03 17:59:55,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0
1,49831911074,Spread 3B,2022-05-03 17:57:32,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0
2,49831911074,Spread 3B,2022-05-03 17:55:23,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0
3,49831911074,Spread 3B,2022-05-03 17:53:29,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0
4,49831911074,Spread 3B,2022-05-03 17:51:45,2.23,2.13,0.10,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.333333,2017.0,373.10,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704313,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:40:12,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,...,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN
1704314,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:45:11,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,...,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN
1704315,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:50:10,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,...,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN
1704316,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:51:07,1.37,1.48,0.00,OK,2.45,NB,Spread 5B,...,49.24,14.2,0.0,6.0,17-015W,True,617.333333,2013.0,666.72,NaN


In [46]:
travel_t.nunique()

AssetNumber                   274
AssetIdentifier               132
CreatedDate                495632
UnderConstructionTime         576
BaselineTime                  275
DelayTime                     635
State                           2
SegmentLength                 133
TravelDirect                    2
Spread                          7
TrafficImpact                   3
RoadType                        3
TrafficControlSetup             2
CreatedDate:Date              171
CreatedDate:Weekday             7
CreatedDate:DayOfYear         107
CreatedDate:Year                2
CreatedDate:Month               4
CreatedDate:Day                31
CreatedDate:Hour               24
CreatedDate:MinuteInDay      1440
CreatedDate:Season              2
Weather:StationID              12
Weather:Long                   11
Weather:Lat                    12
Weather:Temp                  344
Weather:Precip                 46
Weather:WindSpd                39
Traffic:SiteNo                 16
Traffic:IsTemp

In [25]:
assets = pd.read_csv(os.path.join(input_root_path, 'Recommended_Assets_Merged.csv'))
del assets[assets.columns[0]]
assets = assets.dropna(subset=['Segment Name'])
print(f'NaN:\n{assets.isna().sum()}')
assets

NaN:
Asset Number                0
Segment Name                0
Segment Length              1
Baseline                    2
Direct of Travel            0
Spread                      0
Traffic Impact              2
Road Type                   0
Traffic Control Setup       0
Easting                    22
Northing                   22
Most Recent Activation    131
Cell Coverage             449
UTM Zone                  118
dtype: int64


,Asset Number,Segment Name,Segment Length,Baseline,Direct of Travel,Spread,Traffic Impact,Road Type,Traffic Control Setup,Easting,Northing,Most Recent Activation,Cell Coverage,UTM Zone
0,62134790681,AP 3-B-12-A-1,2.746,1.72,NB,Spread 3B,High,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN,11.0
1,10897197432,AP 3-B-12-A-1,2.746,1.73,SB,Spread 3B,High,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN,11.0
2,12768050215,AP 3-B-13-A-1,2.970,1.92,NB,Spread 3B,High,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN,11.0
3,56678162306,AP 3-B-13-A-1,2.970,1.92,SB,Spread 3B,High,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN,11.0
4,86851077697,AP 3-B-13-A-3,18.000,NaN,NB,Spread 3B,High,Multi-Lane Undivided Roadways,SLAT,346092.86128,5.857159e+06,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,34684428742,T3920.0,4.840,0.00,SB,Spread 5A,High,Multi-Lane Divided,Lane Closure,660846.97920,5.552540e+06,NaN,NaN,NaN
620,69027783597,T4068.3,3.460,0.00,SB,Spread 5B,High,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN,NaN
621,92801985490,T4068.3,3.480,0.00,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN,NaN
622,16764775025,TWS_460042,3.690,2.58,SB,Reactivation,High,Multi-Lane Undivided Roadways,Lane Closure,356159.86762,5.876918e+06,4/4/2022,No Cell Coverage Area,11.0


In [26]:
weather_all = pd.DataFrame()

# for station_id in df_matched_weather_ids['Weather:StationID'].unique():
for folder_name in os.listdir(weather_path):
    if not folder_name.isnumeric():
        print(f'!! {folder_name} ignored.')
        continue
    station_id = int(folder_name)
    weather_joined_files = glob.glob(os.path.join(weather_path, f'{station_id}/*.csv'))
    weather_station = load_csv_to_df(weather_joined_files)

    weather_station.insert(loc=0, column='StationID', value=station_id)
    weather_station['Date/Time (LST)'] = pd.to_datetime(weather_station['Date/Time (LST)'])
    # df_weather_station['Date'] = df_weather_station['Date/Time (LST)'].dt.date
    weather_station.insert(loc=weather_station.columns.get_loc('Date/Time (LST)')+1, column='Date', value= weather_station['Date/Time (LST)'].dt.date)
    weather_station.insert(loc=weather_station.columns.get_loc('Time (LST)')+1, column='Hour', value= weather_station['Date/Time (LST)'].dt.hour)

    weather_all = pd.concat([weather_all, weather_station[['StationID', 'Longitude (x)', 'Latitude (y)', 'Date', 'Hour', 'Temp (°C)', 'Precip. Amount (mm)', 'Wind Spd (km/h)']]], axis=0)

weather_all.rename(columns = {'Temp (°C)':'Temp',
                             'Precip. Amount (mm)':'Precip',
                             'Wind Spd (km/h)':'WindSpd',
                             'Longitude (x)':'Long',
                             'Latitude (y)':'Lat'},
                  inplace=True)
weather_all.columns = ['Weather:' + str(col) for col in weather_all.columns]
weather_all = weather_all.drop_duplicates()
weather_all

!! daily 702 ignored.
!! Recommended Assets Merged.csv ignored.
!! SelectedWeatherStations_50KM.xlsx ignored.
!! Station Inventory EN.csv ignored.
!! WeatherInfo_Command_Lines_EN.txt ignored.
!! WeatherStationsInfo.zip ignored.


,Weather:StationID,Weather:Long,Weather:Lat,Weather:Date,Weather:Hour,Weather:Temp,Weather:Precip,Weather:WindSpd
0,1022,-120.80,50.11,2020-01-01,0,NaN,NaN,NaN
1,1022,-120.80,50.11,2020-01-01,1,NaN,NaN,NaN
2,1022,-120.80,50.11,2020-01-01,2,NaN,NaN,NaN
3,1022,-120.80,50.11,2020-01-01,3,NaN,NaN,NaN
4,1022,-120.80,50.11,2020-01-01,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
26299,998,-121.02,50.47,2022-09-30,19,NaN,NaN,NaN
26300,998,-121.02,50.47,2022-09-30,20,NaN,NaN,NaN
26301,998,-121.02,50.47,2022-09-30,21,NaN,NaN,NaN
26302,998,-121.02,50.47,2022-09-30,22,NaN,NaN,NaN


In [27]:
weather =  weather_all.dropna(subset=['Weather:Temp', 'Weather:Precip', 'Weather:WindSpd']).copy()
# weather =  weather_all.dropna(subset=['Weather:Temp']).copy().reset_index(drop=True)
# weather = weather.dropna(subset=['Weather:Temp', 'Weather:Precip', 'Weather:WindSpd'])
weather['Weather:Date'] = pd.to_datetime(weather['Weather:Date'])
weather

,Weather:StationID,Weather:Long,Weather:Lat,Weather:Date,Weather:Hour,Weather:Temp,Weather:Precip,Weather:WindSpd
0,10223,-118.03,52.93,2020-01-01,0,-2.0,0.2,4.0
1,10223,-118.03,52.93,2020-01-01,1,-2.1,0.2,4.0
2,10223,-118.03,52.93,2020-01-01,2,-2.1,0.3,5.0
3,10223,-118.03,52.93,2020-01-01,3,-2.3,0.0,6.0
4,10223,-118.03,52.93,2020-01-01,4,-2.3,0.4,4.0
...,...,...,...,...,...,...,...,...
23875,8214,-119.29,52.13,2022-06-21,19,12.9,0.6,4.0
23876,8214,-119.29,52.13,2022-06-21,20,12.5,0.4,3.0
23877,8214,-119.29,52.13,2022-06-21,21,12.3,1.0,2.0
23878,8214,-119.29,52.13,2022-06-21,22,11.9,0.7,4.0


In [28]:
travel_t_na = travel_t[travel_t[['Weather:Temp', 'Weather:Precip', 'Weather:WindSpd']].isna().any(axis=1)]
# travel_t.dropna(subset=['Weather:Temp', 'Weather:Precip', 'Weather:WindSpd'])
travel_t_na

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,...,Weather:Lat,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:CalculatedAADT,MaxSpeed
41177,98626596943,"AP 3-B-17-A, AP 3-B-Hwy 5-3, AP 3-B-Hwy 5-4 • ...",2021-05-14 21:00:11,3.45,2.53,0.92,OK,3.337,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,126.0,2017.0,131.04,90.0
41178,98626596943,"AP 3-B-17-A, AP 3-B-Hwy 5-3, AP 3-B-Hwy 5-4 • ...",2021-05-14 21:05:12,2.85,2.53,0.32,OK,3.337,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,126.0,2017.0,131.04,90.0
41179,98626596943,"AP 3-B-17-A, AP 3-B-Hwy 5-3, AP 3-B-Hwy 5-4 • ...",2021-05-14 21:10:12,2.77,2.53,0.24,OK,3.337,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,126.0,2017.0,131.04,90.0
41180,98626596943,"AP 3-B-17-A, AP 3-B-Hwy 5-3, AP 3-B-Hwy 5-4 • ...",2021-05-14 21:15:13,2.78,2.53,0.25,OK,3.337,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,126.0,2017.0,131.04,90.0
41181,98626596943,"AP 3-B-17-A, AP 3-B-Hwy 5-3, AP 3-B-Hwy 5-4 • ...",2021-05-14 21:17:15,2.77,2.53,0.24,OK,3.337,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,126.0,2017.0,131.04,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704013,55858030442,AP 6-A-Hwy 1 E-2 • NB • Macro Spiecapag JV • 1052,2021-04-15 18:15:10,1.33,1.27,0.06,OK,2.160,NB,Spread 5B,...,49.36,NaN,NaN,NaN,P-17-11E,NaN,NaN,NaN,NaN,100.0
1704014,55858030442,AP 6-A-Hwy 1 E-2 • NB • Macro Spiecapag JV • 1052,2021-04-15 18:20:10,1.33,1.27,0.06,OK,2.160,NB,Spread 5B,...,49.36,NaN,NaN,NaN,P-17-11E,NaN,NaN,NaN,NaN,100.0
1704015,55858030442,AP 6-A-Hwy 1 E-2 • NB • Macro Spiecapag JV • 1052,2021-04-15 18:25:11,1.28,1.27,0.01,OK,2.160,NB,Spread 5B,...,49.36,NaN,NaN,NaN,P-17-11E,NaN,NaN,NaN,NaN,100.0
1704016,55858030442,AP 6-A-Hwy 1 E-2 • NB • Macro Spiecapag JV • 1052,2021-04-15 18:30:09,1.28,1.27,0.01,OK,2.160,NB,Spread 5B,...,49.36,NaN,NaN,NaN,P-17-11E,NaN,NaN,NaN,NaN,100.0


In [34]:
def find_nearest_ws(asset, wss):
    dists = []
    for ws in wss:
        # dists.append(np.linalg.norm(asset - ws, axis=1)[0])
        dists.append(np.linalg.norm(asset - ws))
    # print('dists', dists, '\n')
    return np.argmin(dists)


# def convert_coordinates(easting, northing, utm_zone):
def convert_coordinates(asset_info):
    print('\n\n', asset_info)
    easting = asset_info['Easting']
    northing = asset_info['Northing']
    utm_zone = asset_info['UTM Zone']

    if np.isnan(easting) or np.isnan(northing) or np.isnan(utm_zone):
        # print("!!! DANGER")
        # print(easting, northing)
        return np.nan

    if pd.isna(utm_zone):
        if easting > 600000:
            utm_zone = 10
        elif easting < 600000:
            utm_zone = 11
        else:
            # if np.isnan(easting) or np.isnan(northing) or np.isnan(utm_zone):
            #     print("!!! DANGER")
            # print(easting, northing)
            # print(f"utm_zone = {utm_zone} \n")
            return np.nan

    utm_zone = int(utm_zone)

    proj_lonlat_to_EN = pyproj.Proj(proj='utm', zone=utm_zone, ellps='WGS84')

    # ea , no = proj_lonlat_to_EN(lon, lat)
    lon, lat = proj_lonlat_to_EN(easting, northing, inverse=True)

    return [lon , lat]

In [32]:
assets['Longitude_Latitude'] = assets.apply(convert_coordinates, axis=1)
assets



 Asset Number                                62134790681
Segment Name                              AP 3-B-12-A-1
Segment Length                                    2.746
Baseline                                           1.72
Direct of Travel                                     NB
Spread                                        Spread 3B
Traffic Impact                                     High
Road Type                 Multi-Lane Undivided Roadways
Traffic Control Setup                              SLAT
Easting                                    344630.96633
Northing                                  5861839.09318
Most Recent Activation                        2/25/2022
Cell Coverage                                       NaN
UTM Zone                                           11.0
Name: 0, dtype: object


 Asset Number                                10897197432
Segment Name                              AP 3-B-12-A-1
Segment Length                                    2.746
Baseline           

,Asset Number,Segment Name,Segment Length,Baseline,Direct of Travel,Spread,Traffic Impact,Road Type,Traffic Control Setup,Easting,Northing,Most Recent Activation,Cell Coverage,UTM Zone,Longitude_Latitude
0,62134790681,AP 3-B-12-A-1,2.746,1.72,NB,Spread 3B,High,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN,11.0,"[-119.30910111899807, 52.883778480895074]"
1,10897197432,AP 3-B-12-A-1,2.746,1.73,SB,Spread 3B,High,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN,11.0,"[-119.30910111899807, 52.883778480895074]"
2,12768050215,AP 3-B-13-A-1,2.970,1.92,NB,Spread 3B,High,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN,11.0,"[-119.30432219789675, 52.875482049886536]"
3,56678162306,AP 3-B-13-A-1,2.970,1.92,SB,Spread 3B,High,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN,11.0,"[-119.30432219789675, 52.875482049886536]"
4,86851077697,AP 3-B-13-A-3,18.000,NaN,NB,Spread 3B,High,Multi-Lane Undivided Roadways,SLAT,346092.86128,5.857159e+06,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,34684428742,T3920.0,4.840,0.00,SB,Spread 5A,High,Multi-Lane Divided,Lane Closure,660846.97920,5.552540e+06,NaN,NaN,NaN,NaN
620,69027783597,T4068.3,3.460,0.00,SB,Spread 5B,High,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN,NaN,NaN
621,92801985490,T4068.3,3.480,0.00,NB,Spread 5B,High,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN,NaN,NaN
622,16764775025,TWS_460042,3.690,2.58,SB,Reactivation,High,Multi-Lane Undivided Roadways,Lane Closure,356159.86762,5.876918e+06,4/4/2022,No Cell Coverage Area,11.0,"[-119.14458716604344, 53.02244478139078]"


In [38]:
# type(asset_info[['Easting', 'Northing']].values)
counter = 0
i = 0
for idx in travel_t_na.index:
    if i % 50000 == 0:
        print(f'{i / len(travel_t_na) * 100}% checked!!')
    obs = travel_t.loc[idx]

    if np.isnan(obs['Weather:Temp']) or np.isnan(obs['Weather:Precip']) or np.isnan(obs['Weather:WindSpd']):
        # print('idx =', idx)
        # print('BEFORE', obs[['Weather:Temp', 'Weather:Precip', 'Weather:WindSpd']])

        weather_time_matched = weather[
            (obs['CreatedDate:Date'] == weather['Weather:Date']) &
            (obs['CreatedDate:Hour'] == weather['Weather:Hour'])]

        asset_info = assets[obs['AssetNumber'] == assets['Asset Number']]

        weather_coordinates = weather_time_matched[['Weather:Lat', 'Weather:Long']]

        nearest_idx = find_nearest_ws(asset_info['Longitude_Latitude'].values[0], weather_time_matched[['Weather:Long', 'Weather:Lat']].values)
        # print(nearest_idx)
        nearset_ws = weather_time_matched.iloc[nearest_idx]


        travel_t.loc[idx, 'Weather:Temp'] = nearset_ws['Weather:Temp']
        travel_t.loc[idx, 'Weather:Precip'] = nearset_ws['Weather:Precip']
        travel_t.loc[idx, 'Weather:WindSpd'] = nearset_ws['Weather:WindSpd']

        # print('\n\nAFTER', travel_t.loc[idx][['Weather:Temp', 'Weather:Precip', 'Weather:WindSpd']])

        counter += 1
    i += 1
counter

0.0% checked!!
5.293374706350038% checked!!
10.586749412700076% checked!!
15.880124119050116% checked!!
21.173498825400152% checked!!
26.46687353175019% checked!!
31.760248238100232% checked!!
37.05362294445027% checked!!
42.346997650800304% checked!!
47.64037235715035% checked!!
52.93374706350038% checked!!
58.22712176985042% checked!!
63.520496476200464% checked!!
68.8138711825505% checked!!
74.10724588890054% checked!!
79.40062059525057% checked!!
84.69399530160061% checked!!
89.98737000795064% checked!!
95.2807447143007% checked!!


421290

In [39]:
travel_t.isna().sum()

AssetNumber                     0
AssetIdentifier                 0
CreatedDate                     0
UnderConstructionTime           0
BaselineTime                   22
DelayTime                      22
State                          85
SegmentLength                   0
TravelDirect                    0
Spread                          0
TrafficImpact                   0
RoadType                        0
TrafficControlSetup             0
CreatedDate:Date                0
CreatedDate:Weekday             0
CreatedDate:DayOfYear           0
CreatedDate:Year                0
CreatedDate:Month               0
CreatedDate:Day                 0
CreatedDate:Hour                0
CreatedDate:MinuteInDay         0
CreatedDate:Season              0
Weather:StationID               0
Weather:Long                    0
Weather:Lat                     0
Weather:Temp                    0
Weather:Precip                  0
Weather:WindSpd                 0
Traffic:SiteNo                  0
Traffic:IsTemp

In [40]:
travel_t.to_csv(os.path.join(input_root_path, 'TravelTime_HaveCellCoverage_WeatherFillNaN.csv'))

In [ ]:
# obs = travel_t_na.loc[1704012]
#
# weather_time_matched = weather[
#     (obs['CreatedDate:Date'] == weather['Weather:Date']) &
#     (obs['CreatedDate:Hour'] == weather['Weather:Hour'])]
# weather_time_matched
#
# asset_info = assets[obs['AssetNumber'] == assets['Asset Number']]
# asset_coordinate = asset_info[['Easting', 'Northing']]
#
# # asset_coordinate
# # weather_time_matched[['Weather:Long', 'Weather:Lat']]
#
# nearest_idx = find_nearest_ws(asset_coordinate.values, weather_time_matched[['Weather:Long', 'Weather:Lat']].values)
# print(nearest_idx)
# weather_time_matched.iloc[nearest_idx]
# # obs['AssetNumber']